### Conditional expression for one latent element

For Gibbs sampling, we need the conditional probability of one latent element $z_{i,k}$, conditional on the other latent elements, and also on the observed data values.

ie, we need:

$$
P(z_{i,k} \mid \mathbf{X}, \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)
$$

Note that since we are marginalizing over $\mathbf{A}$, there is no $\mathbf{A}$ in this expression.

By Bayes Rules, the conditional probability for $z_{i,k}$ is:

$$
P(z_{i,k} | \mathbf{X}, \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)
=
\frac
  {P(\mathbf{Z}_{-(i,k)} \mid z_{i,k}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{X}, \sigma_X, \sigma_A)}
  {P(\mathbf{Z}_{-(i,k)} \mid \mathbf{X}, \sigma_X, \sigma_A)}
$$

$$
=
\frac
  {P(\mathbf{Z}_{-(i,k)} \mid z_{i,k}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k})}
  {P(\mathbf{Z}_{-(i,k)}}
$$

or, we can try slightly differently:

$$
P(z_{i,k} | \mathbf{X}, \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)
=
\frac
  {P(\mathbf{X} \mid z_{i,k}, \mathbf{Z}_{-(i,k)}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{Z}_{-(i,k)},\sigma_X, \sigma_A)}
  {P(\mathbf{X} \mid \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)}
$$

$$
=
\frac
  {P(\mathbf{X} \mid \mathbf{Z}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{Z}_{-(i,k)},\sigma_X, \sigma_A)}
  {P(\mathbf{X} \mid \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)}
$$

$$
\propto P(\mathbf{X} \mid \mathbf{Z}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{Z}_{-(i,k)},\sigma_X, \sigma_A)
$$

(since the denominator is constant wrt $z_{i,k}$, therefore doesnt change the shape of the probability distribution of $z_{i,k}$, simply acts as a constant of normalization)

$P(\mathbf{X} \mid \mathbf{X}, \sigma_X, \sigma_A)$, we've just calculated, by marginalizing the joint probability of $\mathbf{X}$ and $\mathbf{A}$ over $\mathbf{A}$.

$P(z_{i,k} \mid \mathbf{Z}_{-(i,k)})$, we calculated in the previous section, for Gibbs sampling of a finite feature model:

$$
P(z_{i,k} \mid \mathbf{z}_{-i,k})
= \frac{m_{-i,k} + \frac{\alpha}{K}}
  {N + \frac{\alpha}{K}}
$$


### Interlude: rank-1 updates

The formula for the exponential trace expression of the marginalized form over $\mathbf{A}$ needs the calculation of an inverse:

$$
\left(
  \mathbf{Z}^T\mathbf{Z}
  + \frac{\sigma_X^2}{\sigma_A^2}
  \mathbf{I}
\right)^{-1}
$$

Matrix inversions are generally expensive, computationally. For example, if a matrix is symmetric and positive-definite, one can use Cholesky Decomposition to calculate the inverse, which uses about $\frac{1}{3}n^3$ flops, where $n$ is the size of the matrix.  This is less than LU decomposition, which uses about $\frac{2}{3}n^3$ flops, but is still cubic in $n$.

Therefore if we can avoid re-calculating an inverse repeatedly during an algorithmic implementation, then the implementation will run faster.

One way to avoid calculating the inverse repeatedly, is to use rank-1 updates, where this is possible.

Given the Cholesky decomposition of matrix $\mathbf{A}$ into $\mathbf{L}\mathbf{L}^T$, where $\mathbf{L}$ is a lower triangular matrix, then the rank-1 update calculates $\mathbf{L}'$ where:

$$
\mathbf{A} + \mathbf{x}\mathbf{x}^T
= \mathbf{L}' \mathbf{L}'^T
$$

...given vector $\mathbf{x}$, and the original decomposition of $\mathbf{A} = \mathbf{L}\mathbf{L}^T$

Per "Methods for Modifying Matrix Factorizations", by Gill et al, 1974, rank-1 update to Cholesky factorization can be carried out in $O(n^2)$ flops. (method 'C1').

From a practical, engineering, point of view, it looks like numpy doesnt provide an implementation for rank-1 updates.  LINPACK does, and matlab does, but numpy doesnt.

[Wikipedia](https://en.wikipedia.org/wiki/Cholesky_decomposition) provides a matlab implementation of a rank-1 update.  Let's try implementing it in Python, and then try a `numpy` built-in method (which presumably uses BLAS)

In [15]:
import numpy as np
import math
import time

N = 3000
B = np.random.randn(N, N)
A = B.T.dot(B)
print(A.shape)
print(A[:2,:2])
start = time.time()
L = np.linalg.cholesky(A)
print('time for full rank Cholesky %s seconds' % (time.time() - start))
print(L[:3,:3])

# check
LLT = L.dot(L.T)
assert np.abs(A - LLT).max() < 1e-6

x = np.random.randn(N)
print(x.shape)
print(x[:3])

def update_cholesky(L, x):
    N = x.shape[0]
    for k in range(N):
        r = math.sqrt(L[k, k] * L[k, k] + x[k] * x[k])
        c = r / L[k, k]
        s = x[k] / L[k, k]
        L[k, k] = r
        L[k + 1:, k] = (L[k + 1:, k] + s * x[k + 1:]) / c
        x[k + 1:] = c * x[k + 1:] - s * L[k + 1:, k]

update_cholesky(L, x)
LLT2 = L.dot(L.T)
diff = np.abs(A + x.dot(x.T) - LLT2).max()
print(diff)
assert diff < 1e-8

(3000, 3000)
[[ 2970.41996429    21.31644394]
 [   21.31644394  2942.49052004]]
time for full rank Cholesky 0.3525505065917969 seconds
[[ 54.50155928   0.           0.        ]
 [  0.39111622  54.243318     0.        ]
 [  0.93970653   0.2525172   54.49271706]]
(3000,)
[-0.14441793  0.18327688 -1.23138222]
3007.10831474


AssertionError: 

... fails, not sure why.  But anyway, looking at the Griffiths and Ghahramani tutorial again, it looks like it's not actually using a generic rank-1 Cholesky update: they're designing/deriving their own rank-1 update formulae.  So let's go through that.

### rank-1 updates to marginal probability of $\mathbf{X}$

Reminder: $\mathbf{M}$ is:

$$\mathbf{M}
= \left(
  \mathbf{Z}^T\mathbf{Z} + \frac{\alpha_X^2}{\alpha_A^2}
  \mathbf{I}
\right)^{-1}
$$

Per the tutorial, we should define:

$$
\mathbf{M}_{-i}
= \left(
\sum_{j \ne i}
z_j^T z_j
+ \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}
\right)
^{-1}
$$

And then, I cant quite guess/see what the tutorial is/is going to do, so I'm just going to work through what the tutorial does:

$$
\mathbf{M}_{-i}
= (\mathbf{M}^{-1} - \mathbf{z}_i^T\mathbf{z}_i)^{-1}
$$

$$
= ( \dots
$$

I stared at this for a while, and then decided the tutorial must probably be using a generic, well-known, rank-1 update formula.  I googled for 'matrix inverse rank-1 update', and found:

https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula

This gives the following formula:

$$
\left(
  \mathbf{A} + \mathbf{u}\mathbf{v}^T 
\right)^{-1}
=
\mathbf{A}^{-1}
-
\frac{\mathbf{A}^{-1}\mathbf{u}\mathbf{v}^T\mathbf{A}^{-1}}
  {1 + \mathbf{v}^T\mathbf{A}^{-1}\mathbf{u}}
$$

This looks pretty similar to equation (23) in the tutorial, so let's work with the idea that the tutorial is using the Sherman-Morrison formula for now.  Let's first modify the Sherman-Morrison formula in two ways:

- write it for a downdate, ie for $(\mathbf{A} - \dots)^{-1}$ rather than $(\mathbf{A} + \dots)^{-1}$, and
- modify to write for the case that there is only one vector $\mathbf{v}$, rather than the more general form of two unequal vectors $\mathbf{u}$ and $\mathbf{v}$

We can do this by substituting $\mathbf{u} = - \mathbf{v}$:

$$
(\mathbf{A} - \mathbf{v}\mathbf{v}^T)^{-1}
=
\mathbf{A}^{-1}
-
\frac
  {\mathbf{A}^{-1} \cdot (-\mathbf{v}) \cdot \mathbf{v}^T \cdot \mathbf{A}^{-1}}
  {1 + \mathbf{v}^T \cdot \mathbf{A}^{-1} \cdot (-\mathbf{v})}
$$

$$
=
\mathbf{A}^{-1}
+
\frac
  {\mathbf{A}^{-1} \cdot (\mathbf{v}) \cdot \mathbf{v}^T \cdot \mathbf{A}^{-1}}
  {1 - \mathbf{v}^T \cdot \mathbf{A}^{-1} \cdot (\mathbf{v})}
$$

Or, by comparison with what is in the tutorial, giving some idea of where this might be going, reversing the order of the terms in the denominator, and the sign of the right-hand side:

$$
=
\mathbf{A}^{-1}
-
\frac
  {\mathbf{A}^{-1} \cdot (\mathbf{v}) \cdot \mathbf{v}^T \cdot \mathbf{A}^{-1}}
  {\mathbf{v}^T \cdot \mathbf{A}^{-1} \cdot (\mathbf{v}) - 1}
$$

$$
=
\mathbf{A}^{-1}
-
\frac
  {\mathbf{A}^{-1} \mathbf{v} \mathbf{v}^T \mathbf{A}^{-1}}
  {\mathbf{v}^T \mathbf{A}^{-1} \mathbf{v} - 1}
$$

This kind of matches what's in the tutorial, except that we have inverses here, like $\mathbf{A}^{-1}$, whereas the tutorial has $\mathbf{M}$, but since $\mathbf{M}$ is in fact defined as an inverse, ie $(\mathbf{Z}^T\mathbf{Z} - \frac{\alpha_X^2}{\alpha_A^2}\mathbf{I})^{-1}$, then this might work out.



Let's substitute in the terms/notation for our actual concrete expression, into the modified Sherman-Morrison formula.  This gives:

$$

$$

If it was using the Sherman-Formula formula, then, using the tutorial's notation, we'd have:

$$
(\mathbf{M}^{-1} - \mathbf{z}_i^T \mathbf{z}_i)^{-1}
= \mathbf{M}^{-1}
+ 
$$